In [1]:
import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
#import plotly.graph_objs as go
#import plotly.offline as py

In [2]:
# matplotlib 한글 출력..
import matplotlib as mpl
import matplotlib.font_manager as fm
mpl.rcParams['axes.unicode_minus'] = False
path = 'C:/Windows/Fonts/malgun.ttf'  # 내가 원하는 폰트로 따로 설정이 가능
font_name = fm.FontProperties(fname=path, size=50).get_name()
plt.rc('font', family=font_name)

In [3]:
# MySQL Connection 연결
def player_name_call():
    conn = pymysql.connect(host='49.247.132.235', user='kbo_db', password='1234', db='kbo_data_db', charset='utf8')
    # Connection 으로부터 Cursor 생성
    curs = conn.cursor()
    sql = '''select * from kbo_pitcher_player 
            union all
            select * from kbo_hitter_player;'''   
    curs.execute(sql)
    rows = curs.fetchall()
    df_player_name = pd.DataFrame(rows)
    conn.close()
    df_player_name.columns = ['player_id', 'player_name']
    return df_player_name

In [4]:
def MinMax_Scaler(df_scaler):
    scaler = MinMaxScaler()
    list_scaler_name = df_scaler['player_name'].values
    
    if 'mvp' in df_scaler.columns:
        list_scaler_mvp = df_scaler['mvp'].values
        df_scaler = df_scaler.drop(['player_name', 'mvp'], axis=1)
        df_scaler[df_scaler.columns] = scaler.fit_transform(df_scaler[df_scaler.columns])
        df_scaler['mvp'] = list_scaler_mvp
    else:       
        df_scaler = df_scaler.drop('player_name', axis=1)        
        df_scaler[df_scaler.columns] = scaler.fit_transform(df_scaler[df_scaler.columns])
        
    df_scaler['player_name'] = list_scaler_name
    
    return df_scaler

In [5]:
def hit_pit_sum (df_total_hit_m, df_total_pit_m):
    df_total_hit_m.columns = ['player_id', 'player_year', 'player_team', 'player_AVG', 'player_G', 'player_PA', 'player_AB', 'player_RH', 'player_HH',
       'player_2B', 'player_3B', 'player_HRH', 'player_TB', 'player_RBI', 'player_SB', 'player_CS', 'player_BBH', 
       'player_HBPH', 'player_SOH', 'player_GDP', 'player_SLG', 'player_OBP', 'player_E']
    df_total_pit_m.columns = ['player_id', 'player_year', 'player_team', 'player_ERA', 'player_G', 'player_CG', 'player_SHO', 'player_W', 'player_L', 
       'player_SV', 'player_HLD', 'player_WPCT', 'player_TBF', 'player_IP', 'player_HP', 'player_HRP', 'player_BBP',
       'player_HBPP', 'player_SOP', 'player_RP', 'player_ER']
    df_sum_player = pd.merge(df_total_hit_m, df_total_pit_m, how='outer')

    df_player_name = player_name_call()
    df_sum_player = pd.merge(df_sum_player, df_player_name, on = 'player_id', how = 'left')
    df_sum_player = df_sum_player.fillna('0')

    df_team_rank = pd.read_excel('./data/team_win.xlsx', index_col=0)
    team_year_rank = df_team_rank[df_team_rank[0] == int(df_total_hit_m.player_year[0])]
    df_sum_player['player_team'] = df_sum_player['player_team'].apply(lambda x: team_year_rank[1][x == team_year_rank[2]].values[0])
    df_sum_player = df_sum_player.drop(['player_id', 'player_year'], axis=1)
    df_sum_player = df_sum_player.apply(lambda x: x.replace('-','0'))
    df_sum_player['player_IP'] = df_sum_player['player_IP'].apply(lambda x: x.replace(' 1/3', '.3') if(' 1/3' in x) else (x.replace(' 2/3', '.6') if(' 2/3' in x) else x))
    df_sum_player['player_IP'] = df_sum_player['player_IP'].apply(lambda x: x.replace('1/3', '0.3') if('1/3' in x) else (x.replace('2/3', '0.6') if('2/3' in x) else x))

    for i in df_sum_player.columns:
        if not i in ['player_name']:
            df_sum_player[i] = df_sum_player[i].apply(lambda x: float(x))
    df_sum_player = MinMax_Scaler(df_sum_player)

    if df_total_hit_m.player_year[0] != 2020:
        df_mvp_list = pd.read_excel('./data/mvp_list.xlsx', index_col=0)
        df_sum_player['mvp'] = df_sum_player['player_name'].apply(lambda x: 1 if (df_mvp_list.player_name[int(df_total_hit_m.player_year[0])] == x) else 0)
    return df_sum_player

In [6]:
def make_team_df(data_year):
    conn = pymysql.connect(host='49.247.132.235', user='kbo_db', password='1234', db='kbo_data_db', charset='utf8')
    curs = conn.cursor()
    sql = '''SELECT * from kbo_hitter_total_status where player_year = {};'''.format(data_year)   
    curs.execute(sql)
    rows = curs.fetchall()
    df_year_hit = pd.DataFrame(rows)

    curs = conn.cursor()
    sql = '''SELECT * from kbo_pitcher_total_status where player_year = {};'''.format(data_year)    
    curs.execute(sql)
    rows = curs.fetchall()
    df_year_pit = pd.DataFrame(rows)
    conn.close()
    df_year_player = hit_pit_sum(df_year_hit, df_year_pit)
    return df_year_player

In [7]:
def df_train_data (data_year):
    df_train_player = pd.DataFrame()
    for i in range(2010, 2020):
        if i != data_year:
            df_year_player = make_team_df(i)
            df_train_player = pd.concat([df_train_player, df_year_player])
    df_test_player = make_team_df(data_year)
    return df_train_player, df_test_player

In [9]:
df_trian_2019, df_test_2019 = df_train_data(2019)